In [1]:
#Test notebook for tracking rigid body  marker consisting of multiple fiducials within a CT scan volume

#general imports
import copy
import os
import pickle
from tqdm import tqdm

#visualization
import matplotlib.pyplot as plt
from matplotlib import rcParams

#DICOM loading
import pydicom

#clustering and optimization
from scipy import linalg
from scipy.cluster.vq import kmeans, vq
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_gaussian_quantiles
from scipy.optimize import minimize
from skimage import measure
import itertools

#3D data processing
import numpy as np
import open3d as o3d
import transforms3d as t3d
from stl import mesh

#our processing code
from ct_tracking_library.ct_tracking_functions import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
def find_candidate_centroids(target_marker, input_mesh_file = 'temp_mesh.stl', visualize=False):
    '''
    
    input:
    return:
    '''
    marker = np.load('./test_data/'+target_marker+'.npy')
    R =  t3d.euler.euler2mat(np.pi/4+0.1, 0, -np.pi/6-0.2)@t3d.euler.euler2mat(0,0.6,0)# @ t3d.euler.euler2mat(0, np.pi/8, 0)
    marker = (R.T@marker.T).T

    # print("Testing IO for meshes ...")
    mesh = o3d.io.read_triangle_mesh(input_mesh_file)
    mesh.compute_vertex_normals()
    #o3d.visualization.draw_geometries([mesh])

    points = np.array(mesh.vertices)
    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector(points)
    pcd2.estimate_normals()
    #o3d.visualization.draw_geometries([pcd2])

    pcd2 = pcd2.voxel_down_sample(voxel_size = 0.8)
    
    #filter points with further than average distance to neighbors
    cl, ind = pcd2.remove_statistical_outlier(nb_neighbors=10,
                                                    std_ratio=0.8) 
    inlier_cloud = pcd2.select_by_index(ind)
    outlier_cloud = pcd2.select_by_index(ind, invert=True)

    #o3d.visualization.draw_geometries([inlier_cloud])
    
    #----------------------------------------------
    #----------------------------------------------
    #----------------------------------------------
    
    
    #cluster pointcloud
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
        inlier_cloud.cluster_dbscan(eps=2.5, min_points=10, print_progress=True))
    print(labels.max() )
    
    #----------------------------------------------
    #----------------------------------------------
    #----------------------------------------------
    
    max_label = labels.max()  

    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    inlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
    inlier_cloud.estimate_normals()
    #o3d.visualization.draw_geometries([inlier_cloud])
    
    #----------------------------------------------
    #----------------------------------------------
    #----------------------------------------------
    
    good_inds = []
    centroids = []
    pointcloud_ball_clusters = []
    import tqdm

    for i in tqdm.tqdm(range(labels.max()+1)):
        #i = 442
        #print('current label: {}'.format(i))
        selected_indices = np.where(labels==i)
        pcd_selected = inlier_cloud.select_by_index(selected_indices[0])



        points = np.array(pcd_selected.points)

        if points.shape[0] < 1000 and points.shape[0] > 20:
            correctX = points[:,0]#np.load('x.npy')
            correctY = points[:,1]#np.load('y.npy')
            correctZ = points[:,2]#np.load('z.npy')

            r, x0, y0, z0, residules = sphereFit(correctX,correctY,correctZ)
            u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
            x=np.cos(u)*np.sin(v)*r
            y=np.sin(u)*np.sin(v)*r
            z=np.cos(v)*r
            x = x + x0
            y = y + y0
            z = z + z0

            #See how good this sphere fit is
            #print(r, x0, y0, z0, residules)
            errors = []
            for j in range(len(correctX)):
                radius_point = ((correctX[j]-x0)**2+(correctY[j]-y0)**2+(correctZ[j]-z0)**2)**0.5
                error = np.abs(radius_point-r)
                errors.append(error)
            errors = np.array(errors)

            mean_error = np.mean(errors)

            #print('Mean Error: {}'.format(np.mean(errors)))
            #print('Radius: {}'.format(r))
            #good_inds.append(i)

            if target_marker == 'marker1':
                r_target = 2.8
            if target_marker == 'marker2':
                r_target = 3.4
                
            diameter_tolerance = 0.5
            sphere_fit_rmse_tolerance = 0.4
            if mean_error < sphere_fit_rmse_tolerance and r > r_target - diameter_tolerance and r < r_target + diameter_tolerance:
                print('Mean Error: {}'.format(mean_error))
                print('Radius: {}'.format(r))
                print('index: {}'.format(i))
                centroids.append(np.array([x0,y0,z0]))
                good_inds.append(i)
                pointcloud_ball_clusters.append(pcd_selected)


    print(good_inds)
    # print(target_marker)
    ## display pointcloud clusters that are a good fit to our target sphere diameter
    #selected_indices = np.where(labels==good_inds[0])
    #pcd_selected = inlier_cloud.select_by_index(selected_indices[0])
    #o3d.visualization.draw_geometries(pointcloud_ball_clusters)

    
    


    centroid_clusters, pcd_centroid_clusters, o3d_selected_cluster_inds = find_centroid_clusters(centroids,good_inds)

    print('pointcloud centroid clusters: {}'.format(pcd_centroid_clusters))
    print('centroid clusters: {}'.format(centroid_clusters))
    print(o3d_selected_cluster_inds)



    
    #----------------------------------------------
    #----------------------------------------------
    #----------------------------------------------
    #swapped good_inds to good_centroids
    good_centroids = o3d_selected_cluster_inds[0]
    selected_indices = np.where(labels==good_centroids[0])
    pcd_selected = inlier_cloud.select_by_index(selected_indices[0])
    #o3d.visualization.draw_geometries([pcd_selected])
    #good_centroid_clusters_list = []
    
    print('centroid clustrs: {}'.format(centroid_clusters))
    good_centroid_clusters = [centroid_cluster for centroid_cluster in centroid_clusters if len(centroid_cluster) >= marker.shape[0]]
    print('MEEE good centroid clusters: {}'.format(good_centroid_clusters))

    for good_centroids in o3d_selected_cluster_inds: #changed 6/22/2022
        #good_centroids = o3d_selected_cluster_inds[0]     #use the first cluster, perhaps not always the right one????? CHANGED 6/22/2022

        #good_centroid_clusters_list.append(good_centroid_clusters)
        #print(good_centroid_clusters)
        #print(good_centroids)

        #----------------------------------------------
        #----------------------------------------------
        #----------------------------------------------

        #o3d.visualization.draw_geometries([inlier_cloud])


        print('Good centroids: {}'.format(good_centroids))
        if len(good_centroids) > 0:
            print('MULTIPLE GOOD CENTROIDS')
            for i in range(len(good_centroids)): #removed -1 condition!!!!!!!!!! August 16th 2021, not tested with old code...
                selected_indices = np.where(labels==good_centroids[i])
                print('adding cluster: {}'.format(i))
                pcd_selected += inlier_cloud.select_by_index(selected_indices[0]) #this adds the full cluster of points. index 0 is to get rid of the list wrapper

    pcd_selected.paint_uniform_color([0.8, 0.0, 0.8])

    o3d.visualization.draw_geometries([pcd_selected])


    print("Downsample the point cloud with a voxel of 0.05")
    #downpcd = pcd_selected.voxel_down_sample(voxel_size=5.0)
     #----------------------------------------------
    #----------------------------------------------
    #----------------------------------------------
  
       
    
    #THIS IS ONLY USING A SINGLE CENTROID> ASSUMES I DON"T FIND FALSE POSITVES! IF I DO I SHOULD ALL OF THEM IF MULTIPLE. 
    #Also, if cluster size is too big I should run through all the subcluster combinations!

    #find the coordinates of the rigid body in this frame, this can be used as an input for a rigid body solver easily if there are no errors
    
    #### NEXT 2 lines commented on out on 6/22 as not used
    #base_centroid = good_centroid_clusters[0][0] #not using the filtered good centroids right now! should be careful on making sure to choose individual local marker clusters incase we have false marker-rigid-body positives
    #print('Chosen base centroid: {}'.format(base_centroid))


    #for i in range(len(good_centroid_clusters[0])): #CHANGED ON 6/22/2022 to get all the candidates
    marker_centroid_coordinates_list = []


    #print('good centroid clusters: {}'.format(good_centroid_clusters))
    marker_centroid_coordinates = []

    for i in range(len(good_centroid_clusters)):
        local_marker_coordinates = good_centroid_clusters[0][i]# - base_centroid
        marker_centroid_coordinates.append(local_marker_coordinates)

    marker_centroid_coordinates = np.array(marker_centroid_coordinates).squeeze()
    #marker_centroid_coordinates -= marker_centroid_coordinates.mean(axis=0)
    print('centroid coordinates: {}'.format(marker_centroid_coordinates))
    mean_centroid_coordinates = np.mean(np.array(pcd_selected.points), axis=0)
    
    print('mean centroid coodinates: {}'.format(mean_centroid_coordinates))
    print('marker_centroid_coordinates: {}'.format(marker_centroid_coordinates))
    print('marker_centroid_coordinates_list: {}'.format(marker_centroid_coordinates_list))

    coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10.0, origin=marker.mean(axis=0))#mean_centroid_coordinates)
    #mean_cluster_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10.0, origin=marker_centroid_coordinates.mean(axis=0))

    #o3d.visualization.draw_geometries([pcd_selected, coordinate_frame, mean_cluster_coordinate_frame])

    print('marker: {}'.format(marker))
    
    
        #----------------------------------------------
    #----------------------------------------------
    #----------------------------------------------
  
    #o3d.visualization.draw_geometries([pcd_selected, mesh, coordinate_frame, mean_cluster_coordinate_frame])
    
    
    
        #----------------------------------------------
    #----------------------------------------------
    #----------------------------------------------

    
    
        #----------------------------------------------
    #----------------------------------------------
    #----------------------------------------------
  
    #marker_centroid_coordinates1 is defined from a previous run of this same code but renaming the centroid output
    return copy.deepcopy(marker), copy.deepcopy(marker_centroid_coordinates), copy.deepcopy(pcd_selected), copy.deepcopy(mesh), copy.deepcopy(coordinate_frame), np.array(good_centroid_clusters)#, np.array(marker_centroid_coordinates_list)



In [5]:
target_marker = 'marker2'
convert_scan_to_mesh(scan_file='./test_data/bb_marker3.pkl')
marker, marker_centroid_coordinates, pcd_selected, mesh, coordinate_frame, good_centroid_clusters = list(copy.deepcopy(find_candidate_centroids(target_marker = target_marker)))

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 118
117
point cloud has 118 clusters


  0%|          | 0/118 [00:00<?, ?it/s]c:\Users\snowl\Documents\GitHub\ct-fiducial-tracking\ct_tracking_library\ct_tracking_functions.py:80: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  C, residules, rank, singval = np.linalg.lstsq(A,f)
 35%|███▍      | 41/118 [00:00<00:00, 138.76it/s]

Mean Error: 0.1517135102397053
Radius: [3.49048368]
index: 8


100%|██████████| 118/118 [00:00<00:00, 215.33it/s]

Mean Error: 0.11403059109047817
Radius: [3.43469735]
index: 60
Mean Error: 0.10120715917432527
Radius: [3.46287753]
index: 76
Mean Error: 0.0882745904389065
Radius: [3.37092787]
index: 79
Mean Error: 0.37881758504078517
Radius: [3.17051513]
index: 91
[8, 60, 76, 79, 91]
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 2
point cloud has 2 clusters
[0 1 2 3]
[4]
pointcloud centroid clusters: [PointCloud with 4 points., PointCloud with 1 points.]
centroid clusters: [array([[ 43.13876131,  66.50936218, 271.71275486],
       [ 43.34459246,  56.94642245, 277.15401035],
       [ 22.33271577,  64.11314569, 288.65317726],
       [ 33.24582136,  80.87299379, 271.16817587]]), array([[ 25.43722452, 237.50804732, 366.86689564]])]
[[8, 60, 76, 79], [91]]
centroid clustrs: [array([[ 43.13876131,  66.50936218, 271.71275486],
       [ 43.34459246,  56.94642245, 277.15401035],
       [ 22.33271577,  64.11

Downsample the point cloud with a voxel of 0.05
centroid coordinates: [ 43.13876131  66.50936218 271.71275486]
mean centroid coodinates: [ 34.55391825  75.40446209 281.31516973]
marker_centroid_coordinates: [ 43.13876131  66.50936218 271.71275486]
marker_centroid_coordinates_list: []
marker: [[  0.           0.           0.        ]
 [-25.17330897   0.27807421  -0.31303004]
 [ -0.13682961  -1.03970283 -10.9369699 ]
 [-12.25998655  -1.57713288 -23.71379929]]


In [6]:
#display the output from above
o3d.visualization.draw_geometries([pcd_selected, mesh])
print(marker)
print(marker_centroid_coordinates)

[[  0.           0.           0.        ]
 [-25.17330897   0.27807421  -0.31303004]
 [ -0.13682961  -1.03970283 -10.9369699 ]
 [-12.25998655  -1.57713288 -23.71379929]]
[ 43.13876131  66.50936218 271.71275486]


In [7]:
def calculate_transform(num_markers,centroids,marker_geometry, verbose=False):
    '''
    function to find the transfrom info of the input
    input:
    return:
    '''

    centroids = centroids.copy()
    marker_geometry = marker_geometry.copy()
    if verbose:
        print(centroids.shape)
    error = []
    R_list = []
    t_list = []
    permuted_centroids_list = []
    centroids_SE3 = np.vstack((centroids.T, np.ones(num_markers)))
    point_permutations = np.array(list(itertools.permutations(list(range(num_markers)),num_markers)))
    for i in (range(point_permutations.shape[0])):#keep this line
        permuted_centroids = centroids_SE3[:, point_permutations[i]]
        permuted_centroids_list.append(permuted_centroids.copy())
        ret_R, ret_t = rigid_transform_3D(marker_geometry, permuted_centroids[:3,:])
        B2 = (ret_R@marker_geometry) + ret_t
        err = B2 - permuted_centroids[:3,:]
        err = err * err
        err = np.sum(err)
        rmse = np.sqrt(err/len(marker_geometry[0]))
        R_list.append(ret_R)
        t_list.append(ret_t)
        error.append(rmse)
    error = np.array(error)
    min_error = np.amin(error)
    min_error_index = np.argmin(error)
    max_error = np.amax(error)
    max_error_index = np.argmax(error)
    #rint(min_error_index)
    final_R = R_list[min_error_index]
    final_t = t_list[min_error_index]
    permuted_centroids = permuted_centroids_list[min_error_index]

    if verbose:
        if min_error < 10:
            print("Everything looks good!")
            print("the final error is: ",min_error)
        else:
            print("Hmm something doesn't look right ...")
    return final_R,final_t, permuted_centroids, min_error

In [8]:
'''
find best (lowest reconstruction error) marker set in each rigid body (if multiple)
inputs:
returns:
'''
num_markers = marker.shape[0]

R_list = []
t_list = []
permuted_centroids_list = []
error_list = []
for segmented_marker in good_centroid_clusters:
    #segmented_marker = good_centroid_clusters[]

    point_combinations = list(itertools.combinations(list(range(segmented_marker.shape[0])), num_markers))
    #print(len(point_combinations))
    for i in range(len(point_combinations)):
        #print('point combinations: '.format(point_combinations[i]))
        sampled_marker = segmented_marker[point_combinations[i],:]
        #print('sampled marker: {}'.format(sampled_marker))
        final_R, final_t, permuted_centroids, error = calculate_transform(num_markers, sampled_marker, marker.T)
        R_list.append(final_R)
        t_list.append(final_t)
        permuted_centroids_list.append(permuted_centroids)
        error_list.append(error)

error_list = np.array(error_list)
#print(error_list)
#print(R_list)
#print(t_list)
#print(permuted_centroids_list)
min_error = np.amin(error_list)
min_error_index = np.argmin(error_list)
max_error = np.amax(error_list)
max_error_index = np.argmax(error_list)
#rint(min_error_index)
final_R = R_list[min_error_index]
final_t = t_list[min_error_index]
permuted_centroids = permuted_centroids_list[min_error_index]

if min_error < 10:
    print("Everything looks good!")
    print("the final error is: ",min_error)
else:
    print("Hmm something doesn't look right ...")

Everything looks good!
the final error is:  0.12400797901918344


In [9]:

''' 
load marker coordinates and create 3D mesh with coordinate frame for origin using O3D
input:
returns:
'''
marker = np.load('./test_data/'+target_marker+'.npy')
    
R =  t3d.euler.euler2mat(np.pi/4+0.1, 0, -np.pi/6-0.2)@t3d.euler.euler2mat(0,0.6,0)# @ t3d.euler.euler2mat(0, np.pi/8, 0)
marker = (R.T@marker.T).T

print(marker)

marker_3d_base = []
for i in range(marker.shape[0]):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=3.0, resolution=20).translate(marker[i,:]).paint_uniform_color([0., 0., 0.8])
    marker_3d_base.append(copy.deepcopy(sphere))

coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10.0)#.rotate(R, center=(0, 0, 0))#mean_centroid_coordinates)
marker_3d_base.append(coordinate_frame)
o3d.visualization.draw_geometries(marker_3d_base)

[[  0.           0.           0.        ]
 [-25.17330897   0.27807421  -0.31303004]
 [ -0.13682961  -1.03970283 -10.9369699 ]
 [-12.25998655  -1.57713288 -23.71379929]]


In [10]:
'''
reconstruction error checking code
input:
returns:
'''

#https://github.com/dangeo314/ct_imaging_library/blob/main/src/SVD_marker_class.ipynb
np.set_printoptions(2)
transformed_marker = final_R @ marker.T + final_t #transforms the marker, second input to the permuted centroids from first input
print(permuted_centroids.T[:,:3])
print(transformed_marker.T)

err = permuted_centroids.T[:,:3]-transformed_marker.T
print(err)
print(np.linalg.norm(err))

coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10.0)#.rotate(R, center=(0, 0, 0))#mean_centroid_coordinates)
coordinate_frame_transformed = copy.deepcopy(coordinate_frame).rotate(final_R, center=(0, 0, 0)).translate(final_t)

marker_3d_transformed = []
for marker in marker_3d_base:
    marker_3d_transformed.append(copy.deepcopy(marker).rotate(final_R, center=(0, 0, 0)).paint_uniform_color([0.8, 0., 0.]).translate(final_t))

visualization_list = [pcd_selected, mesh, coordinate_frame, coordinate_frame_transformed]+marker_3d_base+marker_3d_transformed

#o3d.visualization.draw_geometries([coordinate_frame_transformed, coordinate_frame])


[[ 43.34  56.95 277.15]
 [ 22.33  64.11 288.65]
 [ 43.14  66.51 271.71]
 [ 33.25  80.87 271.17]]
[[ 43.39  56.96 277.16]
 [ 22.2   64.07 288.76]
 [ 43.22  66.45 271.62]
 [ 33.25  80.96 271.14]]
[[-0.04 -0.01 -0.01]
 [ 0.13  0.04 -0.11]
 [-0.09  0.06  0.09]
 [-0.   -0.09  0.02]]
0.24801595803836687


In [11]:
o3d.visualization.draw_geometries(marker_3d_base+marker_3d_transformed)
o3d.visualization.draw_geometries(visualization_list)